THis notebook contains the code that worked for hamming code eb/n0 plot

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.special import erfc
import HammingCode
import Utils

from scipy.special import erfc

# tamanho da amostra
tamanho = 1e8
variancia = 1

# amostra
amostra_avaliada = np.random.choice([0, 1], size=int(tamanho))

Eb_dB_values = np.arange(-2, 11, 1)

In [ ]:
def decode_sample(data):
    decoded_data = []
    for i in range(0, len(data) - 6, 7):
        decoded_data.extend(HammingCode.decode(data[i:i + 7]).T)
    return decoded_data

def encode_sample(data):
    encoded_data = []
    for i in range(0, len(data) - 3, 4):
        encoded_data.extend(HammingCode.encode(data[i:i + 4]))
    return encoded_data

In [ ]:
def calcular_BER_simulada(amostra, Eb_db):

    amostra_codificada = encode_sample(amostra)
    ruido = np.random.normal(0, np.sqrt(variancia/2), size=int(len(amostra_codificada)))
    
    for i in range(len(amostra_codificada)):
        if amostra_codificada[i] == 0.0:
            amostra_codificada[i] = -1.0
        else:
            amostra_codificada[i] = 1.0

    Eb = 10**(Eb_db /10)

    amostra_transmitida = np.sqrt(Eb) * np.array(amostra_codificada) + ruido
    
    amostra_recebida = np.sign(amostra_transmitida)
    
    for i in range(len(amostra_recebida)):
        if amostra_recebida[i] == -1:
            amostra_recebida[i] = 0
        else:
            amostra_recebida[i] = 1
    
    amostra_decodificada = decode_sample(amostra_recebida)
    return np.sum(amostra_decodificada != amostra) / len(amostra)
  

In [ ]:
BER_simulada = []
SNRs = []
    
for Eb_db in Eb_dB_values:       
    ber = calcular_BER_simulada(amostra_avaliada, Eb_db)

    #SNRs.append(snr)
    BER_simulada.append(ber)

BER_simulada = np.array(BER_simulada)

# Calcular a BER teórica
Eb_values = 10**(Eb_dB_values / 10)
BER_teorica = 0.5 * erfc(np.sqrt(Eb_values))

In [ ]:
# Plotar as taxas de erro de bit simuladas e teóricas
plt.figure(figsize=(10, 6))
plt.semilogy(Eb_dB_values, BER_simulada, linestyle = '--', label='BER Simulada')
plt.semilogy(Eb_dB_values, BER_teorica, linestyle='--', label='BER Teórica')
plt.xlabel('Eb/No (dB)')
plt.ylabel('BER')
plt.grid(True, which='both')
plt.legend()
plt.title('BER Simulada vs BER Teórica')
plt.show()